# 02 Performance Analysis

本章我们介绍一些常用的Python 性能分析的工具和方法。

## 1. Measure computation time

首先，我们讨论一下如何衡量一个函数的运行时间。

我们已菲波纳西数列为例，函数如下。

In [2]:
def fibonacci(n): 
    if n<0: 
        print("Incorrect input") 
    # First Fibonacci number is 0 
    elif n==1: 
        return 0
    # Second Fibonacci number is 1 
    elif n==2: 
        return 1
    else: 
        return fibonacci(n-1) + fibonacci(n-2) 

### 方法1: 使用`time.time()` 函数

This is a quick and dirty mothod.

In [3]:
import time

In [4]:
t_start = time.time()
result = fibonacci(35)
t_end = time.time()

print('result: ', result)
print('time consumption: ', t_end - t_start)

result:  5702887
time consumption:  3.197543144226074


In [5]:
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t_start = time.time()
        result = fn(*args, **kwargs)
        t_end = time.time()
        
        print('@timefn: {} took {} seconds.'.format(fn.__name__, str(t_end - t_start)))
        return result
    
    return measure_time

### 方法2: 使用修饰器

In [6]:
@timefn
def fibonacci_2(n): 
    if n<0: 
        print("Incorrect input") 
    # First Fibonacci number is 0 
    elif n==1: 
        return 0
    # Second Fibonacci number is 1 
    elif n==2: 
        return 1
    else: 
        return fibonacci(n-1) + fibonacci(n-2) 

In [7]:
t_start = time.time()
result = fibonacci_2(35)
t_end = time.time()

print('result: ', result)
print('time consumption: ', t_end - t_start)

@timefn: fibonacci_2 took 3.2178399562835693 seconds.
result:  5702887
time consumption:  3.218092918395996


### 方法3: `timeit`

`python -m timeit -n 5 -r 5 -s "import julia1" "julia1.calc_pure_python(desired_width=1000, max_iterations=300)"`

- `-n` numner, #time of statement
- `-r` repeat, #time to repeat 

执行r次，每次执行n遍

在notebook 中可以直接使用`%timeit` 执行。

In [10]:
%timeit -n 5 -r 3 fibonacci_2(35)

@timefn: fibonacci_2 took 3.148715019226074 seconds.
@timefn: fibonacci_2 took 3.094666004180908 seconds.
@timefn: fibonacci_2 took 3.0998528003692627 seconds.
@timefn: fibonacci_2 took 3.1102662086486816 seconds.
@timefn: fibonacci_2 took 3.101901054382324 seconds.
@timefn: fibonacci_2 took 3.0955021381378174 seconds.
@timefn: fibonacci_2 took 3.1246137619018555 seconds.
@timefn: fibonacci_2 took 3.091104030609131 seconds.
@timefn: fibonacci_2 took 3.1242449283599854 seconds.
@timefn: fibonacci_2 took 3.106576919555664 seconds.
@timefn: fibonacci_2 took 3.0907037258148193 seconds.
@timefn: fibonacci_2 took 3.1099021434783936 seconds.
@timefn: fibonacci_2 took 3.105059862136841 seconds.
@timefn: fibonacci_2 took 3.1034622192382812 seconds.
@timefn: fibonacci_2 took 3.1010360717773438 seconds.
3.11 s ± 3.8 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


同时，我们可以在程序执行的时候对资源进行监测，如下图所示，我们看到了CPU 的变化。

![](./figures/cpu.png)

### 方法4: 使用UNIX 操作系统的time 命令

使用 `-p` 开关，可以得到三个结果：
- real: 整体耗时 
- user: CPU 花在任务上的时间
- sys: 内河函数耗费时间

![](./figures/time.png)

一般说来，user + sys = real，上图中极小的差值可能是花费在IO等待上。

在mac 电脑上，可以安装gnu-time package:

`brew install gnu-time`

然后我们可以使用以下命令查看更多信息：

`gtime --verbose python xxx.py`

类似于下面的结果：

```bash
$ /usr/bin/time --verbose python julia1_nopil.py
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 12.3145110607 seconds
    Command being timed: "python julia1_nopil.py"
    User time (seconds): 13.46
    System time (seconds): 0.05
    Percent of CPU this job got: 99%
    Elapsed (wall clock) time (h:mm:ss or m:ss): 0:13.53
    Average shared text size (kbytes): 0
    Average unshared data size (kbytes): 0
    Average stack size (kbytes): 0
    Average total size (kbytes): 0
    Maximum resident set size (kbytes): 131952
    Average resident set size (kbytes): 0
    Major (requiring I/O) page faults: 0
    Minor (reclaiming a frame) page faults: 58974
    Voluntary context switches: 3
    Involuntary context switches: 26
    Swaps: 0
    File system inputs: 0
    File system outputs: 1968
    Socket messages sent: 0
    Socket messages received: 0
    Signals delivered: 0
    Page size (bytes): 4096
    Exit status: 0
```

比较关键的指标是：

```Major (requiring I/O) page faults: 0```